# Cross Validation & Performance Evaluation

In [1]:
import torch
import numpy as np
import sklearn
import matplotlib.pyplot as plt  
import json
from pathlib import Path

In [2]:
%matplotlib inline

## Collecting data

In [3]:
#Models that are available for testing:
!ls data/

blacklist.txt		    moco32_phase75   simclr_phase25
class_occurences.json	    moco64	     simclr_phase50
gallery_paths_relative.txt  moco64_phase0    simclr_phase75
gallery_paths.txt	    moco64_phase25   strict_train_test.json
imgnet_pretrained	    moco64_phase50   swav
jigsaw			    moco64_phase75   swav_phase0
jigsaw_phase0		    random	     swav_phase25
jigsaw_phase100		    rotnet	     swav_phase50
jigsaw_phase25		    rotnet_phase0    swav_phase75
jigsaw_phase50		    rotnet_phase100  total_cross_val_log_mlp.txt
jigsaw_phase75		    rotnet_phase25   total_cross_val_log_svm.txt
moco32			    rotnet_phase50   total_gallery_mAP_scores_log.txt
moco32_phase0		    rotnet_phase75   total_perf_eval_log_mlp.txt
moco32_phase25		    simclr	     total_perf_eval_log_svm.txt
moco32_phase50		    simclr_phase0


In [4]:
#options rotnet, jigsaw, simclr, moco32, imgnet_pretrained
model_name = "random" #<----------Specifiy model here!

dir = Path("data/" + model_name)

embedding_gallery = torch.load(dir / "embedding_gallery.torch")
embedding_gallery_norm = torch.load(dir / "embedding_gallery_norm.torch")
labels = list()
with open(dir / "embedding_gallery_labels.txt", "r") as f:
    labels = f.read().splitlines()
print(f"Data for model {model_name} succesfully read.\nembedding_gallery size: {embedding_gallery.shape}\
\nembedding_gallery_norm size: {embedding_gallery_norm.shape}\nlabels length: {len(labels)}")

Data for model random succesfully read.
embedding_gallery size: torch.Size([1643, 2048])
embedding_gallery_norm size: torch.Size([1643, 2048])
labels length: 1643


In [5]:
#Convert to numpy arrays:
embedding_gallery = embedding_gallery.numpy()
embedding_gallery_norm = embedding_gallery_norm.numpy()
labels = np.array(labels)
print(f"embedding_gallery size: {embedding_gallery.shape} and type {type(embedding_gallery)}\
\nembedding_gallery_norm size: {embedding_gallery_norm.shape} and type {type(embedding_gallery_norm)}\
\nLabels with size {labels.shape} and type {type(labels)}")

embedding_gallery size: (1643, 2048) and type <class 'numpy.ndarray'>
embedding_gallery_norm size: (1643, 2048) and type <class 'numpy.ndarray'>
Labels with size (1643,) and type <class 'numpy.ndarray'>


## Modelling

In [6]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Let's test the performance of a support vector machine (SVM) and a multi layer perceptron (MLP) on the entire data:

In [ ]:
#Create SVM estimator 
svm_overfit = SVC(kernel="poly")

In [ ]:
#Create MLP estimator
mlp_overfit = MLPClassifier(
    hidden_layer_sizes=(256, 64),
    solver='lbfgs',
    max_iter=10_000
)

In [7]:
#the dat to train/fit on 
data = embedding_gallery_norm.copy()
#labels are already in a numpy array called "labels"

Training/fitting on the data and labels:

In [ ]:
#train SVM
svm_overfit.fit(data, labels)

In [ ]:
#train MLP
mlp_overfit.fit(data, labels)

Evaluate on the same data (this is overfitting, but should yield high results and verify that the data is correct)

In [ ]:
#evaluate SVM
accuracy = svm_overfit.score(data, labels) 
print(f"Accuracy svm: {accuracy*100:.1f}%\n")

In [ ]:
#evaluate MLP
accuracy = mlp_overfit.score(data, labels) 
print(f"Accuracy mlp: {accuracy*100:.1f}%\n")

## cross validation

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit

In [9]:
def cross_validate(classifier, fold, data, labels):
    """
    Performs a cross validation for the classifier with a train test split
    according to fold generator.
    
    Args:
        classifier: the classifier to cross validate.
        fold: the generator to use for train-test splits.
        data: the embedding gallery to train/test on.
        labels: the ground truth labels per row for the gallery.
        
    Returns:
        The mean accuracy score from the cross validation rounds and
        the standard deviation of the scores.
    """
    #cross validation generator
    cv = fold.split(data, y=labels)

    #calc cross validation
    scores = cross_val_score(
        classifier,   #the estimator/classifier
        data,         #data 
        y=labels,     #targets
        cv=cv         #generator for array indicis in data that select a certain split out of "data"
    )
    print(scores)
    accuracy = scores.mean()
    std = scores.std()
    return accuracy, std

Now let's create an svm and mlp that we don't overfit. We will test it's ability to generalize by performing cross validation.

In [ ]:
#Create SVM estimator 
svm_cross = SVC(kernel="poly")

In [ ]:
#Create MLP estimator
mlp_cross = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    solver='lbfgs',
    max_iter=10_000
)

Create cross validation generators for train-test splits:

In [ ]:
iterators = { 
    "KFold" : KFold(n_splits=5), 
    "StratifiedKFold" : StratifiedKFold(n_splits=5), 
    "ShuffleSplit" : ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
}

for k in iterators.keys():
    print(f"Generator/iterator used: {k}")
    iterator = iterators[k]
    for i, (train_index, test_index) in enumerate(iterator.split(data, y=labels)):
        print(f"Fold {i}:")
        print(f"  Train: index[:5] {train_index[:5]} with shape{train_index.shape}")
        print(f"  Test:  index[:5] {test_index[:5]} with shape{test_index.shape}")
    print()

In [ ]:
#Create KFold generator
#5 folds with 1/5 test data and 4/5 training data to cross validate
kf = KFold(n_splits=5)

In [59]:
#Create StratifiedKFold generator
#5 folds with 1/5 test data and 4/5 training data to cross validate. 
#The folds have equal class distribution in this case
skf = StratifiedKFold(n_splits=5)

In [ ]:
#Create ShuffleSplit generator
#samples are shuffled and then split up in a test and train set
ss = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

Perform the cross validations on svm and mlp

### SVM cross validation

In [ ]:
#with KFold
accuracy, std = cross_validate(svm_cross, kf, data, labels)
print(f"Support Vector Machine (SVM) with KFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

In [ ]:
#with StratifiedKFold
accuracy, std = cross_validate(svm_cross, skf, data, labels)
print(f"Support Vector Machine (SVM) with StratifiedKFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

In [ ]:
#with ShuffleSplit
accuracy, std = cross_validate(svm_cross, ss, data, labels)
print(f"Support Vector Machine (SVM) with ShuffleSplit:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

### MLP cross validation

In [ ]:
#with KFold
accuracy, std = cross_validate(mlp_cross, kf, data, labels)
print(f"Multi Layer Perceptron (MLP) with KFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

In [ ]:
#with StratifiedKFold
accuracy, std = cross_validate(mlp_cross, skf, data, labels)
print(f"Multi Layer Perceptron (MLP) with StratifiedKFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

In [ ]:
#with ShuffleSplit
accuracy, std = cross_validate(mlp_cross, ss, data, labels)
print(f"Multi Layer Perceptron (MLP) with ShuffleSplit:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

## Performance Assessment 
In this section we will do the final test.
- Split up the data from cornershop in train (80%) and test (20%) data.
- Cross validate on the training set.
- Fit on the training set.
- Test on the testing set.

In [10]:
from sklearn.metrics import confusion_matrix

In [11]:
def eval_performance(classifier, train_set, train_labels, test_set, test_labels):
    """
    Evaluate performance for the given estimator/classifier.
    Performance is measured by:
        -Training on the train_set and train_labels
        -Measuring accuracy for classification on the test_set with the test_labels
        -Calculate a confusion matrix
    
    Args:
        classifier : the classifier to evaluate.
        train_set (np.ndarray): the data to train on.
        train_labels (np.ndarray): ground truth for training data.
        test_set (np.ndarray): the data to test on.
        test_labels (np.ndarray): ground truth for testing data.
        
    Returns:
        accuracy : accuracy score from predictions on test set.
        cm: The confusion matrix.
    """
    #train on training set
    classifier.fit(train_set, train_labels) #retrain on training set
    
    #Calculate accuracy
    accuracy = classifier.score(test_set, test_labels) #calc acc score on testing set
    
    #Calculate confusion matrix
    y_true = test_labels.copy() #Ground truth
    y_pred = classifier.predict(test_set) #Predictions made by classifier
    proba = classifier.predict_proba(test_set)
    cls = classifier.classes_
    print(f"shape proba is {proba.shape}\n examples {proba[:4]}")
    print(f"sgapr classes {cls.shape}, examples {cls[:4]}")
    print(f"shape test is {test_set.shape}\n examples {test_set[:4]}")
    print(f"amount of classes in test set {len(np.unique(test_labels))}\n")
    print(f"shape y_pred is {y_pred.shape}\n examples {y_pred[:4]}")
    #compute confusion matrix
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    #predicted class is the column nr, ground truth is the row nr
    #correct classifications can be found on the diagonal

    return accuracy, cm

### Train test split

In [12]:
train_set, test_set, train_labels, test_labels= train_test_split(
    data,          #data
    labels,        #targets
    test_size=0.2, #20% test set, 80% train set
    random_state=0 #for reproducable results of the random shuffling
)

In [13]:
#our training set looks like this
print(f"train set\n{train_set[:5]} \n\ntrain_labels \n{train_labels[:5]}")

train set
[[0.04078872 0.03530824 0.00678304 ... 0.03306351 0.01726185 0.03610783]
 [0.04064573 0.0363175  0.00599212 ... 0.03440046 0.01788658 0.03525334]
 [0.0424205  0.03523001 0.00580871 ... 0.0335947  0.01733422 0.03480563]
 [0.04106166 0.0362539  0.00684298 ... 0.03334849 0.01874477 0.03872367]
 [0.04268859 0.03532639 0.00594792 ... 0.0351269  0.01742237 0.03658667]] 

train_labels 
['MayTeaFramboos1L' 'MonsterPipelinePunch500ml' 'AquariusOrangeFles'
 'MonsterUltra' 'PepsiMax']


In [14]:
#our test set looks like this
print(f"test set\n{test_set[:5]} \n\ntest_labels \n{test_labels[:5]}")

test set
[[0.04198815 0.03486998 0.00615037 ... 0.03400399 0.01776142 0.036527  ]
 [0.04279465 0.03503878 0.00742215 ... 0.03626968 0.0162737  0.03606767]
 [0.04264008 0.03634319 0.00699273 ... 0.03464346 0.01640888 0.03668126]
 [0.04302761 0.03545025 0.00602863 ... 0.03395331 0.01711669 0.03462484]
 [0.04186171 0.03634463 0.00635983 ... 0.03284569 0.01839124 0.03855904]] 

test_labels 
['InnocentOrangeWithBits' 'CapriSun' 'MonsterPunchEnergy500ml'
 'NaluOriginal6x350ml' 'StellaBlik50cl']


In [15]:
#Proportion of training data
prop_train = (len(train_set)/ len(embedding_gallery_norm) ) * 100
#Proportion of testing data
prop_test = (len(test_set)/ len(embedding_gallery_norm) ) * 100
print(f"Size of the training set {len(train_set)}\nThe training set contains {prop_train}% of the data\n")
print(f"Size of the test set {len(test_set)}\nThe test set contains {prop_test}% of the data")

Size of the training set 1314
The training set contains 79.97565429093122% of the data

Size of the test set 329
The test set contains 20.02434570906878% of the data


### Cross validation on the training set

#### Support Vector Machine (SVM)

In [ ]:
#Create SVM estimator 
svm_test = SVC(kernel="poly", probability=True)

In [ ]:
#Create StratifiedKFold generator
#5 folds with 1/5 test data and 4/5 training data to cross validate. 
#The folds have equal class distribution in this case
skf = StratifiedKFold(n_splits=5)

In [ ]:
#Perform the cross validation
accuracy, std = cross_validate(svm_test, skf, train_set, train_labels)
print(f"Support Vector Machine (SVM) with StratifiedKFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

#### Multi Layer Perceptron (MLP)

In [28]:
#Create MLP estimator
mlp_test = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    solver='lbfgs',
    max_iter=10_000,
    alpha=0.1
)

In [29]:
#Create StratifiedKFold generator
#5 folds with 1/5 test data and 4/5 training data to cross validate. 
#The folds have equal class distribution in this case
skf = StratifiedKFold(n_splits=5)

In [30]:
%%timeit -n 1 -r 1 #time cell excecution by running it only once
#Perform the cross validation
accuracy, std = cross_validate(mlp_test, skf, train_set, train_labels)
print(f"Multi Layer Perceptron (MLP) with StratifiedKFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

/home/olivier/Documents/master/mp/.venv/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/olivier/Documents/master/mp/.venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/olivier/Documents/master/mp/.venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in

[0.3460076  0.30798479 0.27756654 0.23574144 0.32824427]
Multi Layer Perceptron (MLP) with StratifiedKFold:
Accuracy: 29.9% 
std_dev: 0.039004224492825805

57min 58s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


/home/olivier/Documents/master/mp/.venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


### Testing on the test set

#### Suport Vector Machine (SVM)

In [ ]:
accuracy, cm = eval_performance(svm_test, train_set, train_labels, test_set, test_labels)
print(f"Support Vector Machine (SVM):\nAccuracy: {accuracy*100:.1f}% \nconfusion_matrix: \n{cm}\n")

#### Multi Layer Perceptron (MLP)

In [31]:
%%timeit -n 1 -r 1 #time cell excecution by running it only once
accuracy, cm = eval_performance(mlp_test, train_set, train_labels, test_set, test_labels)
print(f"Multi Layer Perceptron:\nAccuracy: {accuracy*100:.1f}% \nconfusion_matrix: \n{cm}\n")

shape proba is (329, 384)
 examples [[4.65709577e-12 1.07960457e-05 7.78955650e-13 ... 1.22585036e-14
  5.77362896e-18 9.84162609e-07]
 [2.15507052e-25 3.57265629e-21 1.77578006e-16 ... 4.40905041e-22
  6.53800761e-20 1.10062926e-29]
 [8.16649973e-10 5.83621811e-11 2.79276050e-06 ... 1.29708713e-13
  2.53487470e-11 1.14303914e-13]
 [9.99729143e-04 1.99477710e-09 1.51196594e-02 ... 9.07805973e-28
  3.61220578e-27 2.74375173e-09]]
sgapr classes (384,), examples ['7upFree' '7upLemon' '7upMojito' 'AADrink']
shape test is (329, 2048)
 examples [[0.04198815 0.03486998 0.00615037 ... 0.03400399 0.01776142 0.036527  ]
 [0.04279465 0.03503878 0.00742215 ... 0.03626968 0.0162737  0.03606767]
 [0.04264008 0.03634319 0.00699273 ... 0.03464346 0.01640888 0.03668126]
 [0.04302761 0.03545025 0.00602863 ... 0.03395331 0.01711669 0.03462484]]
amount of classes in test set 194

shape y_pred is (329,)
 examples ['InnocentOrangeWithBits' 'CapriSun' 'ChaudfontaineSensation'
 'NaluOriginal6x350ml']
Multi La

/home/olivier/Documents/master/mp/.venv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Strict Peformance Assessment 
In this section we will do the final test with a sticter test set.
This means that every embedding in the test set is from a different origin image than every embedding from the same class in the train set.
- Collect the split from a json file which aims to achieve a 80/20 train/test split.
- Cross validate on the strict training set.
- Fit on the strict training set.
- Test on the strict testing set.

### Train test split

In [ ]:
json_file = Path("data/strict_train_test.json")
# Read the JSON file
with open(json_file, "r") as file:
    json_data = file.read()

# Convert JSON to dictionary
strict_train_test = json.loads(json_data)

# Access the dictionary
print(f"Amount of classes that can be used for strict testing: {len(strict_train_test.keys())} / {len(np.unique(labels))}")

Lets set up the strict test and train set <br>
We have a copy of embedding gallery norm in `data` and the corresponding labels are in the variable `labels`. <br>
We have the indicis of embeddings in the gallery already determined in the json file, let's apply them to get our train and test set:

In [ ]:
train_set_strict = []
train_labels_strict = [] 
test_set_strict = []
test_labels_strict = []
for cls in strict_train_test.keys():
    #Collect train queries
    train_queries = strict_train_test[cls]["train"]
    for query in train_queries:
        idx = query["gallery_idx"]
        train_query = data[idx]
        train_label = labels[idx]
        train_set_strict.append(train_query)
        train_labels_strict.append(train_label)
    
    #Collect test queries
    test_queries = strict_train_test[cls]["test"]
    for query in test_queries:
        idx = query["gallery_idx"]
        test_query = data[idx]
        test_label = labels[idx]
        test_set_strict.append(test_query)
        test_labels_strict.append(test_label)
        

In [ ]:
train_set_strict = np.array(train_set_strict)
test_set_strict = np.array(test_set_strict)

In [ ]:
#our training set looks like this
print(f"train set\n{train_set_strict[:5]} \n\ntrain_labels \n{train_labels_strict[:5]}")

In [ ]:
#our test set looks like this
print(f"test set\n{test_set_strict[:5]} \n\ntest_labels \n{test_labels_strict[:5]}")

In [ ]:
#Proportion of training data
prop_train = (len(train_set_strict)/ (len(train_set_strict) + len(test_set_strict)) ) * 100
#Proportion of testing data
prop_test = (len(test_set_strict)/ (len(train_set_strict) + len(test_set_strict)) ) * 100
print(f"Size of the training set {len(train_set_strict)}\nThe training set contains {prop_train}% of the data\n")
print(f"Size of the test set {len(test_set_strict)}\nThe test set contains {prop_test}% of the data")

### Cross validation on the training set

#### Support Vector Machine (SVM)

In [ ]:
#Create SVM estimator 
svm_test_strict = SVC(kernel="poly")

In [ ]:
#Create StratifiedKFold generator
#5 folds with 1/5 test data and 4/5 training data to cross validate. 
#The folds have equal class distribution in this case
skf = StratifiedKFold(n_splits=5)

In [ ]:
#Perform the cross validation
accuracy, std = cross_validate(svm_test_strict, skf, train_set_strict, train_labels_strict)
print(f"Support Vector Machine (SVM) with StratifiedKFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

#### Multi Layer Perceptron (MLP)

In [ ]:
#Create MLP estimator
mlp_test_strict = MLPClassifier(
    hidden_layer_sizes=(256, 64),
    solver='lbfgs',
    max_iter=10_000
)

In [ ]:
#Create StratifiedKFold generator
#5 folds with 1/5 test data and 4/5 training data to cross validate. 
#The folds have equal class distribution in this case
skf = StratifiedKFold(n_splits=5)

In [ ]:
#Perform the cross validation
accuracy, std = cross_validate(mlp_test_strict, skf, train_set_strict, train_labels_strict)
print(f"Multi Layer Perceptron (MLP) with StratifiedKFold:\nAccuracy: {accuracy*100:.1f}% \nstd_dev: {std}\n")

### Testing on the strict test set

#### Support Vector Machine (SVM)

In [ ]:
accuracy, cm = eval_performance(
    svm_test_strict, 
    train_set_strict, 
    train_labels_strict, 
    test_set_strict, 
    test_labels_strict
)
print(f"Support Vector Machine (SVM):\nAccuracy: {accuracy*100:.1f}% \nconfusion_matrix: \n{cm}\n")

#### Multi Layer Perceptron (MLP)

In [ ]:
accuracy, cm = eval_performance(
    mlp_test_strict, 
    train_set_strict, 
    train_labels_strict, 
    test_set_strict, 
    test_labels_strict
)
print(f"Multi Layer Perceptron:\nAccuracy: {accuracy*100:.1f}% \nconfusion_matrix: \n{cm}\n")